In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import pandas as pd
import preprossesing as pre

list_of_model shape: (152,)
list_of_gearbox_type shape: (3,)
list_of_fuel_type shape: (4,)


python(2752) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [2]:
df = pd.read_csv('data/train.csv')
df = df.drop(['id'], axis=1)
df = pre.encoder(df)

y = df['price'].values
X = df.drop(['price'], axis=1)
X = X.values


KeyError: "['manufacturer'] not found in axis"

In [ ]:
param_grid = {
    'n_estimators': [250, 300, 450],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [20, 30, 40],
    'min_samples_split': [4, 5, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}

# Instantiate the Random Forest model
rf = RandomForestRegressor()

# Set up Grid Search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
# Fit the model using Grid Search
grid_search.fit(X, y)

Fitting 3 folds for each of 324 candidates, totalling 972 fits
[CV] END bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   0.0s
[CV] END bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   0.0s
[CV] END bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   0.0s
[CV] END bootstrap=True, max_depth=20, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=400; tota

/opt/anaconda3/envs/pytorch2.0/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
324 fits failed out of a total of 972.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
318 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/pytorch2.0/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/envs/pytorch2.0/lib/python3.11/site-packages/sklearn/base.py", line 1467, in wrapper
    estimator._validate_params()
  File "/opt/anaconda3/envs/pytorch2.0/lib/python3.11/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_par

GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True, False], 'max_depth': [20, 30, 40],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [200, 300, 400]},
             verbose=2)

In [ ]:
# Get the best parameters and evaluate the model
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X)
mse = mean_squared_error(y, y_pred)
print(f'Best Parameters: {grid_search.best_params_}')
print(f'Mean Squared Error: {mse}')


import pickle

df = pd.read_csv('data/test.csv')
df = pd.read_csv('data/test.csv')

index = df['id']
df = df.drop(['id'], axis=1)
X = pre.encoder(df)
X = pre.standardize(X, scaler=pickle.load(open('scaler.pkl', 'rb')))
X = X.values
y_pred = best_rf.predict(X)

df = pd.DataFrame(y_pred, columns=['price'])
df = pd.concat([index, df], axis=1)
df.rename(columns={'price': 'answer'}, inplace=True)
df.to_csv('output.csv', index=False)

Best Parameters: {'bootstrap': False, 'max_depth': 40, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}
Mean Squared Error: 1586749.9955426322
